<a href="https://colab.research.google.com/github/tobytoyin/sec-10q-msc-report/blob/main/2_Dataset_Creation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import pandas as pd 
from datetime import datetime, timedelta
import numpy as np

## load the base
base_dir = "/content/drive/MyDrive/Aston/Term 3/data"
training_dir = '/content/drive/MyDrive/Aston/Term 3/ML-training/datasets/'

## load the set of paths r
with open(f'{base_dir}/file_paths.json', 'r') as f:
  paths = json.load(f)

def read_fin_data(path):
  # load the financial ratio as base dataframe 
  df = pd.read_csv(path, index_col=0)
  df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')
  df = df.fillna(value=np.nan)
  return df

paths

{'10_q_doc_path': '/content/drive/MyDrive/Aston/Term 3/data/datasets/text_data_lg.csv',
 'balsheet_path': '/content/drive/MyDrive/Aston/Term 3/data/datasets/balsheet.csv',
 'base_dir': '/content/drive/MyDrive/Aston/Term 3/data',
 'cashflow': '/content/drive/MyDrive/Aston/Term 3/data/datasets/cashflow.csv',
 'companies_path': '/content/drive/MyDrive/Aston/Term 3/data/selected_companies.csv',
 'filing_original_dir': '/content/drive/MyDrive/Aston/Term 3/data/10-Q-original',
 'filing_records_path': '/content/drive/MyDrive/Aston/Term 3/data/edgar-filing-parsing-full.csv',
 'fin_price_path': '/content/drive/MyDrive/Aston/Term 3/data/datasets/daily_price.csv',
 'fin_ratio_cont': '/content/drive/MyDrive/Aston/Term 3/data/datasets/financial_data_continous.csv',
 'fin_ratio_path': '/content/drive/MyDrive/Aston/Term 3/data/datasets/financial_data.csv',
 'metrics_path': '/content/drive/MyDrive/Aston/Term 3/data/datasets/metrics_data.csv',
 'models_dir': '/content/drive/MyDrive/Aston/Term 3/data/mo

In [ ]:
### Define The Financial Variables In Here ### 
fund_include = [
                'netAssets', 'BP', 'DP', 'currentRatio', 
                'SP', 'CFP', 'ROE', 'ROA', 'roic', 'peRatio', 'pbRatio', 'eps',
                'EquityRatio', 'assetsTurnover', 'marketCap'
                ]

fund_without_pct = ['dividendPayoutRatio', 'dividendYield',
                    'symbol', 'fillingDate', 'period']                

                   ## percentage change only
fund_pct_only = ['revenue', 'totalAssets', 'capitalExpenditure']

In [ ]:
ratio_df = read_fin_data(paths['fin_ratio_path'])
balsheet_df = read_fin_data(paths['balsheet_path'])
metric_df = read_fin_data(paths['metrics_path'])
statement_df = read_fin_data(paths['statement_path'])
cashflow_df = read_fin_data(paths['cashflow'])
tech_ind_df = read_fin_data(paths['tech_ind'])
statement_df['fillingDate'] = pd.to_datetime(statement_df['fillingDate'], format='%Y-%m-%d')
balsheet_df['filingDate'] = pd.to_datetime(balsheet_df['fillingDate'], format='%Y-%m-%d')
cashflow_df['filingDate'] = pd.to_datetime(cashflow_df['fillingDate'], format='%Y-%m-%d')


# # load the stock price data 
stock_df = pd.read_csv(paths['fin_price_path'], header=[0,1], index_col=0)
stock_df.index = pd.to_datetime(stock_df.index, format='%Y-%m-%d')
stock_df = stock_df.fillna(value=np.nan)

snp_df = pd.read_csv(paths['snp_path'], index_col=0)
vix_df = pd.read_csv('/content/drive/MyDrive/Aston/Term 3/data/datasets/vix.csv', index_col=0)

# load the text data
text_df = pd.read_csv(paths['10_q_doc_path'], index_col='acc_num')
text_df['filing_date'] = pd.to_datetime(text_df['filing_date'], format='%Y-%m-%d')

earnings_df = pd.read_csv('/content/drive/MyDrive/Aston/Term 3/data/datasets/earnings_sur.csv', index_col=0)
earnings_df = earnings_df[['Symbol', 'filing_date', 'eps_sur', 'estimate_date', 'earning_sur_pct']]
earnings_df['filing_date'] = pd.to_datetime(earnings_df['filing_date'], format='%Y-%m-%d')
earnings_df['estimate_date'] = pd.to_datetime(earnings_df['estimate_date'], format='%Y-%m-%d')

In [ ]:
# earnings_df

In [ ]:
## Handle Merging similar DataFrames 
cols_to_use = metric_df.columns.difference(ratio_df.columns)
cols_to_use = [*cols_to_use, 'date', 'symbol']

fundamental_df = pd.merge(ratio_df, metric_df[cols_to_use], 
                          how='left', 
                          left_on=['date', 'symbol'], 
                          right_on=['date', 'symbol'])


cols_to_use = statement_df.columns.difference(fundamental_df.columns)
cols_to_use = [*cols_to_use, 'date', 'symbol']
fundamental_df = pd.merge(fundamental_df, statement_df[cols_to_use], 
                          how='left', 
                          left_on=['date', 'symbol'], 
                          right_on=['date', 'symbol'])

cols_to_use = balsheet_df.columns.difference(fundamental_df.columns)
cols_to_use = [*cols_to_use, 'date', 'symbol']
fundamental_df = pd.merge(fundamental_df, balsheet_df[cols_to_use], 
                          how='left', 
                          left_on=['date', 'symbol'], 
                          right_on=['date', 'symbol'])


cols_to_use = cashflow_df.columns.difference(fundamental_df.columns)
cols_to_use = [*cols_to_use, 'date', 'symbol']
fundamental_df = pd.merge(fundamental_df, cashflow_df[cols_to_use], 
                          how='left', 
                          left_on=['date', 'symbol'], 
                          right_on=['date', 'symbol'])
# list(ratio_columns)

In [ ]:
### Caluclate Some Financial Ratio ###
fundamental_df.fillna(value={'dividendPayoutRatio': 0, 
                   'dividendPaidAndCapexCoverageRatio': 0, 
                   'dividendYield': 0, 
                   'totalCurrentLiabilities': 0, 
                   'longTermDebt': 0, 
                   'payoutRatio': 0}, inplace=True)

fundamental_df['netAssets'] = fundamental_df['totalAssets'] - fundamental_df['totalLiabilities']
fundamental_df['BP'] = fundamental_df['netAssets'] / fundamental_df['marketCap']
fundamental_df['DP'] = fundamental_df['dividendsPaid'] / fundamental_df['marketCap']
fundamental_df['SP'] = fundamental_df['revenue'] / fundamental_df['marketCap']
fundamental_df['CFP'] = fundamental_df['operatingCashFlow'] / fundamental_df['marketCap']
fundamental_df['ROE'] = fundamental_df['netIncome'] / fundamental_df['netAssets']
fundamental_df['ROA'] = fundamental_df['operatingIncome'] / fundamental_df['totalAssets']
fundamental_df['EquityRatio'] = fundamental_df['netAssets'] / fundamental_df['totalAssets']
fundamental_df['assetsTurnover'] = fundamental_df['revenue'] / fundamental_df['totalAssets']
fundamental_df['rd_mve'] = fundamental_df['researchAndDevelopmentExpenses'] / fundamental_df['marketCap']

ratio_columns = fundamental_df.columns.drop(['symbol', 'date', 'period', 'link',
                                             'filingDate', 'finalLink', 'acceptedDate',
                                             'fillingDate', 'reportedCurrency'])

In [ ]:
# separate each stock into a dictionary
stock_df_collections = {}
fundamental_collections = {}

idx = pd.IndexSlice
companies = stock_df.columns.levels[1]

############################# 
### Build Stock DataFrame ###report
#############################
time_index = stock_df.index # align every dataframe time index
for company in companies:
  empty_df = pd.DataFrame(index=stock_df.index)
  # empty_df.index.name = 'Date'
  company_stock = stock_df.loc[:, idx[:, company]].copy()

  # drop level 
  company_stock = company_stock.droplevel(1, axis=1)
  company_stock['symbol'] = company 
  company_stock = empty_df.join(company_stock)
  company_stock.fillna(method='ffill', inplace=True, limit=2)
  company_stock = company_stock.join(snp_df, 
                          how='left', rsuffix='_snp')
  company_stock = company_stock.join(vix_df, 
                          how='left', rsuffix='_vix')
  stock_df_collections[company] = company_stock

############################# 
### Build Financial Ratio ###
#############################
for company in companies:
  company_fund = fundamental_df.loc[fundamental_df['symbol'] == company].copy()

  # replace these with zero 
  # dividend_cols = {'dividendPayoutRatio': 0, 
  #                  'dividendPaidAndCapexCoverageRatio': 0, 
  #                  'dividendYield': 0, 
  #                  'totalCurrentLiabilities': 0, 
  #                  'longTermDebt': 0, 
  #                  'payoutRatio': 0}
  # company_fund.fillna(value=dividend_cols, inplace=True)

  # backfill others by maximum 1 year
  # company_fund.fillna(method='bfill', limit=4, inplace=True)
  company_fund[ratio_columns].fillna(company_fund[ratio_columns].rolling(4, min_periods=1).mean().shift(-3), inplace=True)
  company_fund.set_index('date', inplace=True)

  # store in collection
  fundamental_collections[company] = company_fund


# fundamental_collections['AAPL'].head(30)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [ ]:
def apply_to_companies(variable_name, func):
  global stock_df_collections
  for company in companies:
    df = stock_df_collections[company].copy()
    df[variable_name] = func(df)
    stock_df_collections[company] = df

def apply_to_fundamentals(variable_name, func):
  global fundamental_collections
  for company in companies:
    df = fundamental_collections[company].copy()
    df[variable_name] = func(df)
    fundamental_collections[company] = df

def get_price(df, col, new_shift):
  return df[col].shift(new_shift)

def substract(df, x, y):
  return df[x] - df[y]

def return_rate(df, new, old, new_shift=0, old_shift=1, normalize=True):
  """Calculate the Stock return by lag days

  shift (positive) - past n days
  shift (negative) - future n days
  """

  # stock return 
  df = df.copy()

  new_price = df[new]
  old_price = df[old]

  new_shifted = new_price.shift(new_shift)
  old_shifted = old_price.shift(old_shift)

  stock_return = (new_shifted / old_shifted) - 1

  # snp_return 
  if normalize:
    new_price = df[f"{new}_snp"]
    old_price = df[f"{old}_snp"]

    new_shifted = new_price.shift(new_shift)
    old_shifted = old_price.shift(old_shift)

    snp_return = (new_shifted / old_shifted) - 1
    return stock_return - snp_return

  return stock_return

def volatility(df, window, normalize=True):
  rates = return_rate(df, 'Close', 'Close', 1, 2, normalize)
  result = rates.rolling(window, min_periods=1).std()
  return result

def skewness(df, window, normalize=True):
  rates = return_rate(df, 'Close', 'Close', 1, 2, normalize)
  # snp_rates = return_rate(df, 'Close_snp', 'Close_snp', 1, 2)

  skew = rates.rolling(window, min_periods=1).skew()
  return skew 

  snp_skew = snp_rates.rolling(window, min_periods=1).skew()

  if normalize:
    return skew - snp_skew

  return skew

def simple_moving_average(df, window, by='return_rate', normalize=True):
  if by == 'return_rate':
    rates = return_rate(df, 'Close', 'Close', 1, 2, normalize)
  
  else: 
    rates = get_price(df, by, 1)

  mean = rates.rolling(window, min_periods=1).mean()
  return mean

def relative_strength_index(df, window, shift_start_date=1):
  def sum_gt(x, gt=0):
    return sum(filter(lambda a: a >= gt, x))

  def sum_lt(x, lt=0):
    return sum(filter(lambda a: a < lt, x))

  # find the shift 1 day
  difference = df['Close'].shift(shift_start_date).diff(1)
  average_gain = difference.rolling(window).apply(sum_gt)
  average_loss = difference.rolling(window).apply(sum_lt)

  return 100 - (100 / (1 + (average_gain / -average_loss)))


def chmom(df):
  """Return Month 6 to Month 1 - Month 12 to Month 7"""
  month6_to_1 = return_rate(df, 'Close', 'Close', 31, 210)
  month12_to_7 = return_rate(df, 'Close', 'Close', 210, 360)
  return month6_to_1 - month12_to_7

def ear(df):
  rates = return_rate(df, 'Close', 'Close', 1, 2)
  return rates.rolling(3).sum()

def cal_shift(df, shift):
  return df.shift(shift)

## cash 
def cash_to_avg_assets(df):
  # average assets 
  average_total_assets = df['totalAssets'].rolling(2).mean().shift(-1)
  return df['cashAndCashEquivalents'] / average_total_assets

def nincr(df):
  """Counts Num of increaseing earnings of 8 pervious quarters"""
  window = 8
  def count_consecutive(x):
    count = 0
    for i in x:
      if i == 0:
        return count
      count += 1

    return count
  # Find Diff - if earning increases compare to the past 
  earning_diff = df['netIncome'].diff(-1)
  if_earning_increase = earning_diff > 0
  consecutive_num_increase_earning = (
      if_earning_increase.rolling(window)
      .apply(count_consecutive)
      .shift(-window)
  )

  return consecutive_num_increase_earning

def moving_average_future_return(df, days_after=60, period=5, normalize=True):
  day_5 = return_rate(df, 'Open', 'Close', -period, 1, normalize)
  # day_5 = return_rate(df, 'Open', 'Close', -period -1, -1)

  for i in range(1, int(days_after / period)):
    # print((i+1) * -5, ' --- ', i * -5)
    day_5 += return_rate(df, 'Open', 'Close', (i+1) * -period, i * -period, normalize)

  # print('--------------')
  
  # average = day_5 / 6  
  return day_5


#### Market Drift Period 
def post_filing_volatility(df, window=2, normalize=True):
  rates = return_rate(df, 'Close', 'Close', -1, 1, normalize)
  result = rates.rolling(window).std()
  return result

def post_filing_volumne(df):
  rates = return_rate(df, 'Volume', 'Volume', -1, 1)
  result = rates.rolling(2).mean()
  return result

def post_moving_average_future_return(df, days_after=60, period=5):
  day_5 = return_rate(df, 'Open', 'Open', -period -2, -2)
  # day_5 = return_rate(df, 'Open', 'Close', -period -1, -1)

  for i in range(1, int(days_after / period)):
    # print((i+1) * -period -2, i * -period - 2)
    day_5 += return_rate(df, 'Open', 'Open', (i+1) * -period -2, i * -period - 2)

  # print('--------------')
  
  # average = day_5 / 6  
  return day_5  

In [ ]:
apply_to_companies('y_return30_nom', lambda x: return_rate(x, 'Open', 'Close', -2, 1))
# apply_to_companies('y_return30_snp', lambda x: return_rate(x, 'Open_snp', 'Close_snp', -30, 1))
# apply_to_companies('y_return30_nom', lambda x: substract(x, 'y_return30', 'y_return30_snp'))
# apply_to_companies('y_return30_nom', moving_average_future_return)
# apply_to_companies('return30', lambda x: moving_average_future_return(x, normalize=False))

# market drift within post filing period 
# apply_to_companies('y_return30_nom', post_moving_average_future_return)
# apply_to_companies('post_filing_vol', post_filing_volatility)
# apply_to_companies('post_filing_mom', lambda x: return_rate(x, 'Close', 'Close', -1, 1))  # 1 week
# apply_to_companies('post_filing_vix', lambda x: get_price(x, 'Close_vix', -1))
# apply_to_companies('post_filing_vix_pct', lambda x: return_rate(x, 'Close_vix', 'Close_vix', -1, 1, False))  # 1 week
# apply_to_companies('post_filing_volume', post_filing_volumne)


# get vix index 
apply_to_companies('vix_1', lambda x: get_price(x, 'Close_vix', 1))
apply_to_companies('vix_pct30', lambda x: return_rate(x, 'Close_vix', 'Close_vix', 1, 31, False))  # 1 week
# apply_to_companies('vix_pct60', lambda x: return_rate(x, 'Close_vix', 'Close_vix', 1, 61, False))  # 1 week
# apply_to_companies('vix_pct90', lambda x: return_rate(x, 'Close_vix', 'Close_vix', 1, 91, False))  # 1 week

# volume 
apply_to_companies('volume_7', lambda x: simple_moving_average(x, 7, 'Volume', False))
apply_to_companies('volume_30', lambda x: simple_moving_average(x, 30, 'Volume', False))
apply_to_companies('volume_60', lambda x: simple_moving_average(x, 60, 'Volume', False))
apply_to_companies('volume_90', lambda x: simple_moving_average(x, 90, 'Volume', False))

# price return Momentum
apply_to_companies('mom_7', lambda x: return_rate(x, 'Close', 'Close', 1, 8))  # 1 week
apply_to_companies('mom_30', lambda x: return_rate(x, 'Close', 'Close', 1, 31))  # 1 month
apply_to_companies('mom_60', lambda x: return_rate(x, 'Close', 'Close', 1, 61))  # 2 months 
apply_to_companies('mom_90', lambda x: return_rate(x, 'Close', 'Close', 1, 91))  # 1 quarter 
apply_to_companies('mom_360', lambda x: return_rate(x, 'Close', 'Close', 1, 361))  # 1 year 

# momemtum of SnP
# apply_to_companies('mom_7_snp', lambda x: return_rate(x, 'Close_snp', 'Close_snp', 1, 8, False))  # 1 week
# apply_to_companies('mom_30_snp', lambda x: return_rate(x, 'Close_snp', 'Close_snp', 1, 31, False))  # 1 month

# price return volataility
apply_to_companies('volatility7', lambda x: volatility(x, 7))
apply_to_companies('volatility30', lambda x: volatility(x, 30))
apply_to_companies('volatility60', lambda x: volatility(x, 60))
apply_to_companies('volatility90', lambda x: volatility(x, 90))
apply_to_companies('volatility360', lambda x: volatility(x, 360))

# apply_to_companies('retvol', lambda x: cal_shift(volatility(x, 30), 30))

# skewness 
# apply_to_companies('skew7', lambda x: skewness(x, 7))
# apply_to_companies('skew30', lambda x: skewness(x, 30))
# apply_to_companies('skew60', lambda x: skewness(x, 60))
# apply_to_companies('skew90', lambda x: skewness(x, 90))
# apply_to_companies('skew360', lambda x: skewness(x, 360))

# simple moving average of price returns 
apply_to_companies('sma7', lambda x: simple_moving_average(x, 7))
apply_to_companies('sma30', lambda x: simple_moving_average(x, 30))
apply_to_companies('sma60', lambda x: simple_moving_average(x, 60))
apply_to_companies('sma90', lambda x: simple_moving_average(x, 90))

# Relative Strength Index 
# apply_to_companies('rsi14', lambda x: relative_strength_index(x, 14))
# apply_to_companies('chmom', chmom)
# apply_to_companies('ear', ear)

# ############## Apply to Fundamentals ################
# apply_to_fundamentals('cash_avg_assets', cash_to_avg_assets)
# apply_to_fundamentals('nincr', nincr)

In [ ]:
## Calculate Percentage For Selected Columns 
pct_change_col = fund_include + fund_pct_only

for company in companies:
  company_fund = fundamental_collections[company]
  pct_cols = []
  
  # calcuate pct change
  for col in pct_change_col:
    # annuual percentage change 
    for i in [1, 4]:
      col_name = f'{col}_pct{i}'
      pct_change = company_fund[col].pct_change(-i)

      # consider no percentage change as 0 change 
      pct_change.fillna(0, inplace=True)

      company_fund[col_name] = pct_change
      pct_cols.append(col_name)

  # select features 
  company_fund = company_fund[[*fund_include, *pct_cols, *fund_without_pct]]
  fundamental_collections[company] = company_fund

In [ ]:
full_data = pd.DataFrame()

## Merge Fundamental to Stock Data (base on filing date)
for company in companies:
  fundemental = fundamental_collections.get(company)
  stock_data  = stock_df_collections.get(company)

  # merge fundamental with stock data
  df = pd.merge(fundemental.reset_index(), 
                stock_data.reset_index(), 
                how='inner', 
                left_on=['fillingDate', 'symbol'], 
                right_on=['Date', 'symbol'])
  
  # merge with text data 
  df = pd.merge(df, text_df, 
                how='inner', 
                left_on=['fillingDate', 'symbol'], 
                right_on=['filing_date', 'sym'])

  # shift 1 
  df['return30_prev'] = df['y_return30_nom'].shift(-1)
  
  # combine 
  full_data = pd.concat([full_data, df])

# combine with earnings 
def look_for_earnings(x, offset=0):
  if offset > 60:
    return 0
  
  date = x['fillingDate']
  symbol = x['symbol']

  match_date = earnings_df['estimate_date'] == date - pd.DateOffset(offset) 
  match_symbol = earnings_df['Symbol'] == symbol
  earnings = earnings_df[match_date & match_symbol]['earning_sur_pct']

  if earnings.empty:
    offset += 1
    return look_for_earnings(x, offset)

  try: 
    return earnings.to_numpy()[0]
  except IndexError:
    return 0
  
full_data.shape

full_data['eps_sur'] = full_data.apply(look_for_earnings, axis=1)
print('FULL DATA: ', full_data.shape)

FULL DATA:  (12903, 104)


In [ ]:
removal_data = full_data[full_data['Close'] > 5]
removal_data.dropna(inplace=True)
print('AFTER NA REMOVAL: ', removal_data.shape)

AFTER NA REMOVAL:  (11966, 104)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
removal_data = removal_data.drop(columns=['date', 'Date', 
                                    'fog_index', 'Unnamed: 0',
                                    'Open_snp', 'High_snp', 'Low_snp', 'Close_snp', 'Adj Close_snp', 'Volume_snp',
                                    'Open_vix', 'High_vix', 'Low_vix', 'Close_vix', 'Adj Close_vix', 'Volume_vix',
                                    'Adj Close', 'High', 'Low', 'Open', 'Close'])

# full_data.to_csv(f'{training_dir}/full2_data.csv')
# full_data.to_csv(f'{training_dir}/full3_data.csv')
# full_data.drop(columns=['10-q-text']).to_csv(f'{training_dir}/full_notext.csv')
removal_data.to_csv(f'{training_dir}/full_data_lg2.csv')


# full_data.drop(columns=['10-q-text']).to_csv(f'{base_dir}/full_notext.csv')
# full_data.to_csv(f'{training_dir}/full_notext.csv')

In [ ]:
# ### Testing ###
# company = full_data['sym'] == 'AAPL'
# date = full_data['filing_date'] == pd.Timestamp('2019-07-31')

# full_data.loc[company & date][['y_return30_nom']]

# # fundamental_collections['ACN'][['currentRatio', 'currentRatio_pct']]
# full_data.shape

In [ ]:
for col in full_data.columns:
  print(col, ' : ', end='')
  print(np.sum(pd.isna(full_data[col])))

date  : 0
netAssets  : 1
BP  : 1
DP  : 1
currentRatio  : 4
SP  : 1
CFP  : 1
ROE  : 1
ROA  : 1
roic  : 5
peRatio  : 7
pbRatio  : 12
eps  : 0
EquityRatio  : 2
assetsTurnover  : 1
marketCap  : 1
netAssets_pct1  : 0
netAssets_pct4  : 0
BP_pct1  : 0
BP_pct4  : 0
DP_pct1  : 0
DP_pct4  : 0
currentRatio_pct1  : 0
currentRatio_pct4  : 0
SP_pct1  : 0
SP_pct4  : 0
CFP_pct1  : 0
CFP_pct4  : 0
ROE_pct1  : 0
ROE_pct4  : 0
ROA_pct1  : 0
ROA_pct4  : 0
roic_pct1  : 0
roic_pct4  : 0
peRatio_pct1  : 0
peRatio_pct4  : 0
pbRatio_pct1  : 0
pbRatio_pct4  : 0
eps_pct1  : 0
eps_pct4  : 0
EquityRatio_pct1  : 0
EquityRatio_pct4  : 0
assetsTurnover_pct1  : 0
assetsTurnover_pct4  : 0
marketCap_pct1  : 0
marketCap_pct4  : 0
revenue_pct1  : 0
revenue_pct4  : 0
totalAssets_pct1  : 0
totalAssets_pct4  : 0
capitalExpenditure_pct1  : 0
capitalExpenditure_pct4  : 0
dividendPayoutRatio  : 0
dividendYield  : 0
symbol  : 0
fillingDate  : 0
period  : 0
Date  : 0
Adj Close  : 4
Close  : 4
High  : 4
Low  : 4
Open  : 4
Volume  